## Import Libraries

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
import requests
from io import StringIO
import GPy

In [11]:
import numpy
import scipy
import sklearn
print(f"NumPy version: {numpy.__version__}")
print(f"SciPy version: {scipy.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

NumPy version: 1.25.2
SciPy version: 1.11.1
Scikit-learn version: 1.3.0


## Data Load (PIMA)

In [ ]:
def load_pima_dataset_full_features():
    url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
    col_names = ['pregnancies', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
    df = pd.read_csv(url, header=None, names=col_names)
    
    feature_names = col_names[:-1]
    X = df[feature_names].values
    y = df['label'].values
    y = np.where(y == 0, -1, 1)
    
    for col in ['glucose', 'bp', 'skin', 'insulin', 'bmi']:
        col_index = col_names.index(col)
        non_zero_values = X[:, col_index][X[:, col_index] != 0]
        if non_zero_values.size > 0:
            mean_val = non_zero_values.mean()
            X[:, col_index][X[:, col_index] == 0] = mean_val
        else:
            X[:, col_index][X[:, col_index] == 0] = 0 
    
    return X, y, feature_names

## Data Load (CRABS)

In [ ]:
def load_crabs_dataset_all_features():
    url = "https://raw.githubusercontent.com/fernandomayer/data/master/crabs.csv"
    
    df = pd.read_csv(
        url,
        sep=';',           # Use semicolon as the delimiter
        header=0,          # The first row is the header
        decimal=',',       # Use comma as the decimal separator
        quotechar='"',     # Handle double quotes around fields
        engine='python'    # Python engine for robustness with custom separators/quotes
    )

    df = df.rename(columns={'especie': 'sp', 'sexo': 'sex'})

    df['sp'] = df['sp'].astype(str).str.strip()
    df['sex'] = df['sex'].astype(str).str.strip()
    
    print(f"Crabs dataset: Unique 'sp' values before filtering: {df['sp'].unique()}")

    df = df[df['sp'].isin(['azul', 'laranja'])]
    
    if df.empty:
        raise ValueError(
            "DataFrame is empty after filtering 'sp' column. "
            "This indicates 'azul' or 'laranja' species might not be present or are malformed."
        )

    feature_columns = ['FL', 'RW', 'CL', 'CW', 'BD']

    for col in feature_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    X = df[feature_columns].values
    
    if np.isnan(X).any():
        print("Warning: NaNs found in Crabs features. Imputing with mean...")
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
        X = imputer.fit_transform(X)

    y = np.where(df['sp'] == 'azul', 1, -1)
    
    return X, y, feature_columns

## RBF Kernel

In [ ]:
def rbf_kernel_ard(X1, X2, length_scales, sigma_f=1.0):
    length_scales = np.asarray(length_scales).flatten()
    if len(length_scales) != X1.shape[1]:
        raise ValueError("Number of length scales must match number of features.")
    
    sqdist = np.sum((X1[:, np.newaxis, :] - X2[np.newaxis, :, :])**2 / length_scales**2, axis=-1)
    # Augmenting jitter for numerical stability
    K = sigma_f**2 * np.exp(-0.5 * sqdist)
    if X1.shape[0] == X2.shape[0] and np.array_equal(X1, X2):
        K += 1e-4 * np.eye(X1.shape[0]) # Increased jitter
    return K

## Sigmoid Function

In [ ]:
def sigmoid(x):
    x_clamped = np.clip(x, -500, 500) 
    return np.where(x_clamped >= 0, 1 / (1 + np.exp(-x_clamped)), np.exp(x_clamped) / (1 + np.exp(x_clamped)))

## Laplace Approximation For GP

In [ ]:
def laplace_approximation_and_marginal_likelihood(X, y, kernel_func, length_scales, sigma_f):
    n = X.shape[0]
    K = kernel_func(X, X, length_scales, sigma_f)
    
    f = np.zeros(n)
    for _ in range(100): 
        pi = sigmoid(y * f)
        W = pi * (1 - pi)
        
        W[W < 1e-10] = 1e-10 
        W_sqrt = np.sqrt(W)
        
        grad = y * (pi - 1)
        
        try:
            B = np.eye(n) + np.diag(W_sqrt) @ K @ np.diag(W_sqrt)
            L = np.linalg.cholesky(B)
        except np.linalg.LinAlgError:
            K_stable = K + 1e-3 * np.eye(n) # More aggressive jitter if Cholesky fails
            B = np.eye(n) + np.diag(W_sqrt) @ K_stable @ np.diag(W_sqrt)
            L = np.linalg.cholesky(B)

        b = W * f + grad
        
        try:
            v_intermediate = W_sqrt * (K @ b) 
            v_solve = np.linalg.solve(L, v_intermediate) 
            a_term = np.linalg.solve(L.T, v_solve) 
            a = b - W_sqrt * a_term
            
        except np.linalg.LinAlgError as e:
            f_hat_on_error = np.zeros(n) 
            return f_hat_on_error, 1e10, K, W_sqrt, L 

        f_hat = K @ a
        
        if np.linalg.norm(f - f_hat) < 1e-7 or np.any(np.abs(f_hat) > 1e15): 
            break
        f = f_hat
    
    pi_hat = sigmoid(y * f_hat)
    pi_hat[pi_hat <= 0] = 1e-10 
    log_p_y_given_f = np.sum(np.log(pi_hat))
    
    neg_log_marginal_likelihood = 0.5 * f_hat.T @ np.linalg.solve(K, f_hat) + \
                                  np.sum(np.log(np.diag(L))) - \
                                  log_p_y_given_f
    
    return f_hat, neg_log_marginal_likelihood, K, W_sqrt, L

## Prediction with GP

In [ ]:
def predict_gp(X_test, X_train, y_train, K, f_hat, W_sqrt, L, kernel_func, length_scales, sigma_f):
    K_s = kernel_func(X_train, X_test, length_scales, sigma_f)
    
    v = np.linalg.solve(L, W_sqrt[:, np.newaxis] * K_s) 
    
    mu_f_s = K_s.T @ (y_train * (sigmoid(y_train * f_hat) - 1))

    K_ss = kernel_func(X_test, X_test, length_scales, sigma_f)
    
    var_f_s = np.diag(K_ss) - np.sum(v**2, axis=0)
    
    var_f_s[var_f_s < 0] = 0
    
    denominator_term = 1 + np.pi * var_f_s / 8
    prob = sigmoid(mu_f_s / np.sqrt(denominator_term + 1e-10))
    
    return prob

## Classification Function

In [ ]:
def run_classification_pipeline(dataset_loader_func, visualize=False):
    X, y, feature_names = dataset_loader_func()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    dataset_name = dataset_loader_func.__name__.replace("load_", "").replace("_dataset", "").replace("_full_features", "").replace("_all_features", "").title()
    print(f"================== {dataset_name} Dataset ==================")
    print(f"Training on {X_train.shape[0]} samples, testing on {X_test.shape[0]} samples.")
    print(f"Number of features: {X_train.shape[1]}")

    def gp_objective(params):
        min_log_val_length_scales = -6.0
        max_log_val_length_scales = 6.0
        min_log_val_sigma_f = -3.5 # Adjusted to try to find better sigma_f
        max_log_val_sigma_f = 6.0

        length_scales = np.exp(params[:-1])
        sigma_f = np.exp(params[-1])

        if np.any(length_scales < 1e-4) or sigma_f < 1e-4:
             return 1e9

        try:
            f_hat, neg_log_ml, _, _, _ = laplace_approximation_and_marginal_likelihood(
                X_train, y_train, rbf_kernel_ard, length_scales, sigma_f
            )
            if np.all(f_hat == 0) and neg_log_ml > 1e8:
                return 1e9
            if np.isnan(neg_log_ml) or np.isinf(neg_log_ml):
                return 1e9
            return neg_log_ml
        except np.linalg.LinAlgError:
            return 1e9
        except ValueError as e:
            return 1e9

    num_features = X_train.shape[1]
    
    num_restarts = 50
    best_neg_log_ml = np.inf
    best_optimized_params = None

    print("\n--- Training Gaussian Process Model (with restarts) ---")
    print(f"Optimizing GP hyperparameters with {num_restarts} random restarts...")

    search_bounds = [(-6.0, 6.0)] * num_features + [(-3.5, 6.0)]
    optimizer_options = {'maxiter': 2000}

    for i in range(num_restarts):
        random_initial_params = np.random.uniform(-4.0, 4.0, num_features + 1)
        random_initial_params[:-1] = np.clip(random_initial_params[:-1], search_bounds[0][0], search_bounds[0][1])
        random_initial_params[-1] = np.clip(random_initial_params[-1], search_bounds[-1][0], search_bounds[-1][1])

        current_res = minimize(gp_objective, random_initial_params, bounds=search_bounds, 
                               method='L-BFGS-B', options=optimizer_options)
        
        current_neg_log_ml = current_res.fun

        if current_neg_log_ml < best_neg_log_ml:
            best_neg_log_ml = current_neg_log_ml
            best_optimized_params = current_res.x

    if best_optimized_params is None:
        raise RuntimeError("GP optimization failed to find any valid parameters after all restarts.")

    optimized_params = best_optimized_params
    optimized_params[:-1] = np.clip(optimized_params[:-1], search_bounds[0][0], search_bounds[0][1])
    optimized_params[-1] = np.clip(optimized_params[-1], search_bounds[-1][0], search_bounds[-1][1])
    
    optimized_length_scales = np.exp(optimized_params[:-1])
    optimized_sigma_f = np.exp(optimized_params[-1])

    print("GP Optimization complete.")
    print(f"  - Final Optimal sigma_f: {optimized_sigma_f:.3f}")
    for i, ls in enumerate(optimized_length_scales):
        if i < len(feature_names):
            print(f"  - Final Optimal length_scale for '{feature_names[i]}': {ls:.3f}")
        else:
            print(f"  - Final Optimal length_scale {i}: {ls:.3f} (No corresponding feature name)")

    try:
        f_hat, _, K, W_sqrt, L = laplace_approximation_and_marginal_likelihood(
            X_train, y_train, rbf_kernel_ard, optimized_length_scales, optimized_sigma_f
        )
        gp_probs = predict_gp(X_test, X_train, y_train, K, f_hat, W_sqrt, L, rbf_kernel_ard, optimized_length_scales, optimized_sigma_f)
        gp_y_pred = np.where(gp_probs > 0.5, 1, -1)
        gp_accuracy = accuracy_score(y_test, gp_y_pred)
        print(f"\n>>> GP Test Accuracy: {gp_accuracy:.4f} <<<")
    except np.linalg.LinAlgError:
        print("\nGP prediction failed due to LinAlgError. Accuracy cannot be computed.")
        gp_accuracy = np.nan
    except Exception as e:
        print(f"\nAn error occurred during GP prediction: {e}. Accuracy cannot be computed.")
        gp_accuracy = np.nan

    if visualize and X_train.shape[1] == 2:
        x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
        y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
        grid_points = np.c_[xx.ravel(), yy.ravel()]
        
        grid_points_scaled = scaler.transform(grid_points)

        Z = predict_gp(grid_points_scaled, X_train, y_train, K, f_hat, W_sqrt, L, rbf_kernel_ard, optimized_length_scales, optimized_sigma_f)
        Z = Z.reshape(xx.shape)

        plt.figure(figsize=(10, 7))
        contour = plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm, levels=np.linspace(0, 1, 11))
        plt.colorbar(contour, label='GP Probability of Class +1')
        plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
        plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.coolwarm, edgecolors='k', marker='o', s=80, label='Train Data')
        plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.coolwarm, edgecolors='k', marker='x', s=80, label='Test Data')

        plt.xlabel(f"{feature_names[0]} (Standardized)")
        plt.ylabel(f"{feature_names[1]} (Standardized)")
        plt.title(f'GP Classification Results for {dataset_name} Dataset')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    print("\n--- Training Support Vector Machine (SVM) Model for Comparison ---")
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf']
    }
    
    grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=0, cv=5)
    print("Running GridSearchCV for SVM...")
    grid.fit(X_train, y_train)
    
    print("SVM GridSearch complete.")
    print(f"  - Best SVM Parameters: {grid.best_params_}")
    
    svm_y_pred = grid.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_y_pred)
    print(f"\n>>> SVM Test Accuracy: {svm_accuracy:.4f} <<<")
    print("=" * 60 + "\n")

In [18]:
run_classification_pipeline(load_pima_dataset_full_features, visualize=False)
run_classification_pipeline(load_crabs_dataset_all_features, visualize=False)

================== Pima Dataset ==================
Training on 537 samples, testing on 231 samples.
Number of features: 8

--- Training Gaussian Process Model (with restarts) ---
Optimizing GP hyperparameters with 50 random restarts...
GP Optimization complete.
  - Final Optimal sigma_f: 0.030
  - Final Optimal length_scale for 'pregnancies': 0.021
  - Final Optimal length_scale for 'glucose': 403.429
  - Final Optimal length_scale for 'bp': 0.320
  - Final Optimal length_scale for 'skin': 0.012
  - Final Optimal length_scale for 'insulin': 0.002
  - Final Optimal length_scale for 'bmi': 41.546
  - Final Optimal length_scale for 'pedigree': 0.032
  - Final Optimal length_scale for 'age': 66.989

>>> GP Test Accuracy: 0.4935 <<<

--- Training Support Vector Machine (SVM) Model for Comparison ---
Running GridSearchCV for SVM...
SVM GridSearch complete.
  - Best SVM Parameters: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

>>> SVM Test Accuracy: 0.7446 <<<

Crabs dataset: Unique 'sp' value